# Problem: User Confirmation Rate

**Description:**

The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

**Task:**

Write a solution to find the confirmation rate of each user.

**Return the result table in any order.**

**Example:**

**Input:** 
- Signups table:
  ```sql
  +---------+---------------------+
  | user_id | time_stamp          |
  +---------+---------------------+
  | 3       | 2020-03-21 10:16:13 |
  | 7       | 2020-01-04 13:57:59 |
  | 2       | 2020-07-29 23:09:44 |
  | 6       | 2020-12-09 10:39:37 |
  +---------+---------------------+

Confirmations table:
  ```sql
+---------+---------------------+-----------+
| user_id | time_stamp          | action    |
+---------+---------------------+-----------+
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |
+---------+---------------------+-----------+


Output: 
```sql
+---------+-------------------+
| user_id | confirmation_rate |
+---------+-------------------+
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |
+---------+-------------------+

# Problem: Confirmation Rate Calculation

## Intuition
The problem involves calculating a confirmation rate for each user based on their signups and confirmations. We need to handle cases where users might not have any confirmations to avoid division by zero or undefined rates.

### FIRST using IFNULL SUM COUNT solution

#### Intuition
- We need to link signups with their confirmations, considering all users even if they have no confirmations.

#### Approach
1. **Join the Tables**: Use a LEFT JOIN to ensure all users from `Signups` are included.
2. **Count and Sum**: 
   - Count distinct timestamps to avoid overcounting if multiple actions occur at the same time.
   - Sum 'confirmed' actions for numerator calculation.
3. **Rate Calculation**: 
   - Use `IFNULL` to set the rate to 0 if there are no confirmations.
   - Calculate rate as number of confirmed divided by total actions.
4. **Rounding**: Round to two decimal places for clarity.

#### Complexity
- **Time complexity**: $$O(n \log n)$$ due to `GROUP BY`.
- **Space complexity**: $$O(n)$$ for storing joined results.


````SQL
SELECT 
    s.user_id,
        ROUND(
            IFNULL(
                SUM(CASE WHEN c.action = 'confirmed' THEN 1 ELSE 0 END) / 
                COUNT(DISTINCT c.time_stamp),
                0
        ),
    2) AS confirmation_rate
FROM 
    Signups s
LEFT JOIN 
    Confirmations c ON s.user_id = c.user_id
GROUP BY 
    s.user_id;

# SECOND solution using COALESCE and a simple JOIN Confirmation Rate Calculation

# Confirmation Rate Calculation

## Intuition
The primary intuition here is to calculate a rate based on user actions. We start with the idea that we need to count how many times an action was 'confirmed' versus how many times any action (assuming 'timeout' represents any non-confirmation) was requested. We'll use this ratio to determine the confirmation rate for each user, ensuring that users without any confirmations or requests still appear in our results with a rate of 0.

## Approach
- **Start with all users**: Use the `Signups` table to ensure every user is included in the result, even if they have no confirmations.
- **Join with Confirmations**: Use a LEFT JOIN to include all user data from `Signups`, matching with confirmation data where applicable.
- **Calculate Confirmation Rate**:
  - Count 'confirmed' actions with `SUM(c.action='confirmed')` which turns Boolean results into 1/0 for summing.
  - Count all attempts with `COUNT(c.action='timeout')` assuming 'timeout' represents all non-confirmed actions. Here, we might need a rethink if 'timeout' doesn't represent all non-confirmation actions.
  - Divide these to get the rate, round to two decimal places, and use `COALESCE` to return 0 if there's no data to calculate from (avoiding NULL).
- **Group by User**: Since we want a rate per user, we group by `user_id`.

## Complexity
- **Time complexity**: 
O(n+m), where *n* is the number of rows in `Signups` and *m* is the number of rows in `Confirmations`, because we essentially scan each table once and join them. The join operation can be considered linear if indexed properly.
- **Space complexity**: 
  O(n), where *n* is the number of unique users. We're grouping by user, so in the worst case, we store one result row per user.


# Code
````SQL
SELECT s.user_id AS user_id,
  COALESCE(ROUND(SUM(c.action='confirmed')/COUNT(c.action='timeout'),2),0) AS confirmation_rate
FROM Signups s
LEFT JOIN Confirmations c ON c.user_id = s.user_id
GROUP BY s.user_id;
